# Setup
(No need to read)

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
    # Needed for PySvelte to work, v3 came out and broke things...
    %pip install typeguard==2.13.3
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-g08miz33
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-g08miz33
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 10d2f8a026d73eada861c7d51064f7e24d8f482c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━


## Installing the NodeSource Node.js 16.x repo...


## Populating apt-get cache...

+ apt-get update
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [43.3 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,235 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [850 kB]
Get:12 http:

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [3]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
from dataclasses import dataclass
import datasets
from IPython.display import HTML

In [4]:
import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

Plotting helper functions:

In [5]:
import plotly.graph_objects as go

update_layout_set = {"xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat", "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth"}
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    if "facet_labels" in kwargs_pre:
        facet_labels = kwargs_pre.pop("facet_labels")
    else:
        facet_labels = None
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0,labels={"x":xaxis, "y":yaxis}, **kwargs_pre).update_layout(**kwargs_post)
    if facet_labels:
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label

    fig.show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(y=utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

def lines(lines_list, x=None, mode='lines', labels=None, xaxis='', yaxis='', title = '', log_y=False, hover=None, **kwargs):
    # Helper function to plot multiple lines
    if type(lines_list)==torch.Tensor:
        lines_list = [lines_list[i] for i in range(lines_list.shape[0])]
    if x is None:
        x=np.arange(len(lines_list[0]))
    fig = go.Figure(layout={'title':title})
    fig.update_xaxes(title=xaxis)
    fig.update_yaxes(title=yaxis)
    for c, line in enumerate(lines_list):
        if type(line)==torch.Tensor:
            line = utils.to_numpy(line)
        if labels is not None:
            label = labels[c]
        else:
            label = c
        fig.add_trace(go.Scatter(x=x, y=line, mode=mode, name=label, hovertext=hover, **kwargs))
    if log_y:
        fig.update_layout(yaxis_type="log")
    fig.show()

def bar(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.bar(
        y=utils.to_numpy(tensor),
        labels={"x": xaxis, "y": yaxis},
        template="simple_white",
        **kwargs).show(renderer)

In [6]:
import transformer_lens.patching as patching
from transformer_lens import evals
import math

In [7]:
def visualize_attn_patterns(heads, local_tokens, local_cache, title: str = ""):
    labels = []
    patterns = []
    batch_index = 0

    for head in heads:
        if isinstance(head, tuple):
            layer, head_index = head
        else:
            layer, head_index = head // model.cfg.n_heads, head % model.cfg.n_heads
        patterns.append(local_cache["pattern", layer][batch_index, head_index])
        labels.append(f"L{layer}H{head_index}")
    patterns = torch.stack(patterns, dim=-1)
    attn_viz = pysvelte.AttentionMulti(tokens=model.to_str_tokens(local_tokens[batch_index]), attention=patterns, head_labels=labels)
    display(HTML(f"<h3>{title}</h3>"))
    attn_viz.show()

# Load Model

In [8]:
torch.set_grad_enabled(True)

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [10]:
MAX_NUM=200
cfg = HookedTransformerConfig(
    n_layers=1,
    n_heads=1,
    d_model=32,
    d_head=32,
    n_ctx=3, # a b =
    d_vocab=MAX_NUM+1, # 0, 1, ..., MAX_NUM-1, =
    d_vocab_out=MAX_NUM,
    normalization_type=None,
    attn_only=True,
    device=device,
    seed=0
)

model = HookedTransformer(cfg)
print(model)

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): Identity()
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_post): HookPoint()
    )
  )
  (unembed): Unembed()
)


In [11]:
# disable positional embeddings
model.pos_embed.W_pos = nn.Parameter(torch.zeros_like(model.pos_embed.W_pos))
model.pos_embed.W_pos.requires_grad = False

In [12]:
# disable biases
for name, param in model.named_parameters():
    if 'b_' in name:
        param.requires_grad = False
    print(name, param.shape, param.requires_grad)

embed.W_E torch.Size([201, 32]) True
pos_embed.W_pos torch.Size([3, 32]) False
blocks.0.attn.W_Q torch.Size([1, 32, 32]) True
blocks.0.attn.W_K torch.Size([1, 32, 32]) True
blocks.0.attn.W_V torch.Size([1, 32, 32]) True
blocks.0.attn.W_O torch.Size([1, 32, 32]) True
blocks.0.attn.b_Q torch.Size([1, 32]) False
blocks.0.attn.b_K torch.Size([1, 32]) False
blocks.0.attn.b_V torch.Size([1, 32]) False
blocks.0.attn.b_O torch.Size([32]) False
unembed.W_U torch.Size([32, 200]) True
unembed.b_U torch.Size([200]) False


# Training The Model

## Task dataset

In [13]:
a_vec = einops.repeat(torch.arange(MAX_NUM), "i -> (i j)", j=MAX_NUM)
print(a_vec.shape)
print(a_vec[:5])

b_vec = einops.repeat(torch.arange(MAX_NUM), "i -> (j i)", j=MAX_NUM)
print(b_vec.shape)
print(b_vec[:5])

eq_vec = torch.ones_like(a_vec) * cfg.d_vocab-1
print(eq_vec.shape)
print(eq_vec[:5])

dataset = torch.stack([a_vec, b_vec, eq_vec], dim=-1).to(device)
print(dataset.shape)
print(dataset[:5])

torch.Size([40000])
tensor([0, 0, 0, 0, 0])
torch.Size([40000])
tensor([0, 1, 2, 3, 4])
torch.Size([40000])
tensor([200, 200, 200, 200, 200])
torch.Size([40000, 3])
tensor([[  0,   0, 200],
        [  0,   1, 200],
        [  0,   2, 200],
        [  0,   3, 200],
        [  0,   4, 200]], device='cuda:0')


In [14]:
labels = dataset.min(dim=-1).values
print(labels.shape)
print(labels[:5])

torch.Size([40000])
tensor([0, 0, 0, 0, 0], device='cuda:0')


In [15]:
DATA_SEED = 4
torch.manual_seed(DATA_SEED)

train_frac = 0.8

indices = torch.randperm(dataset.shape[0])
cutoff = int(train_frac * dataset.shape[0])

train_indices = indices[:cutoff]
test_indices = indices[cutoff:]

train_data = dataset[train_indices]
train_labels = labels[train_indices]
print(train_data.shape)
print(train_data[:5])
print(train_labels.shape)
print(train_labels[:5])

test_data = dataset[test_indices]
test_labels = labels[test_indices]
print(test_data.shape)
print(test_data[:5])
print(test_labels.shape)
print(test_labels[:5])

torch.Size([32000, 3])
tensor([[  7, 130, 200],
        [182,   0, 200],
        [110, 155, 200],
        [ 82,   8, 200],
        [126,  97, 200]], device='cuda:0')
torch.Size([32000])
tensor([  7,   0, 110,   8,  97], device='cuda:0')
torch.Size([8000, 3])
tensor([[ 19, 171, 200],
        [  6, 195, 200],
        [ 77,   0, 200],
        [ 97,  39, 200],
        [194,  42, 200]], device='cuda:0')
torch.Size([8000])
tensor([19,  6,  0, 39, 42], device='cuda:0')


## Loss fn

In [16]:
def loss_fn(logits, labels):
    if logits.ndim == 3:
        logits = logits[:, -1, :]

    logits = logits.to(torch.float64)
    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels[..., None])
    return -correct_log_probs.mean()

with torch.no_grad():
    train_logits = model(train_data)
    train_loss = loss_fn(train_logits, train_labels)
    print(train_loss)

    test_logits = model(test_data)
    test_loss = loss_fn(test_logits, test_labels)
    print(test_loss)

tensor(5.3118, device='cuda:0', dtype=torch.float64)
tensor(5.3117, device='cuda:0', dtype=torch.float64)


In [17]:
print("uniform loss:", np.log(model.cfg.d_vocab_out))

uniform loss: 5.298317366548036


## Setup Optimizer

In [18]:
lr = 1e-3
wd = 1.0
betas = (0.9, 0.98)

optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd, betas=betas)

## Training Loop

In [19]:
num_epochs = 4000

train_losses = []
test_losses = []
for epoch in tqdm.tqdm(range(num_epochs)):
    train_logits = model(train_data)
    train_loss = loss_fn(train_logits, train_labels)
    train_loss.backward()
    train_losses.append(train_loss.item())

    optimizer.step()
    optimizer.zero_grad()

    with torch.inference_mode():
        test_logits = model(test_data)
        test_loss = loss_fn(test_logits, test_labels)
        test_losses.append(test_loss.item())

    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, train_loss: {train_loss.item()}, test_loss: {test_loss.item()}")

  0%|          | 0/4000 [00:00<?, ?it/s]

Epoch: 0, train_loss: 5.311815715960998, test_loss: 5.3079134446619225
Epoch: 100, train_loss: 4.42157072643788, test_loss: 4.402977924066611
Epoch: 200, train_loss: 1.0696433031596198, test_loss: 1.0552361521768059
Epoch: 300, train_loss: 0.20378159609211002, test_loss: 0.21225169956642273
Epoch: 400, train_loss: 0.0651223712076428, test_loss: 0.0758592472087193
Epoch: 500, train_loss: 0.022508329382729623, test_loss: 0.032558631629622393
Epoch: 600, train_loss: 0.007571175788413922, test_loss: 0.016541677077114507
Epoch: 700, train_loss: 0.002716881626533853, test_loss: 0.010826971184498942
Epoch: 800, train_loss: 0.0010170458793993993, test_loss: 0.008527235750295478
Epoch: 900, train_loss: 0.00039951809477506206, test_loss: 0.007421142258790176
Epoch: 1000, train_loss: 0.00016100406074683884, test_loss: 0.006828076606638773
Epoch: 1100, train_loss: 6.62079704278245e-05, test_loss: 0.006532583899927963
Epoch: 1200, train_loss: 2.784183797310489e-05, test_loss: 0.00634621205379648
Ep

In [20]:
lines(
    [train_losses, test_losses],
    title="Loss curves",
    xaxis="Epoch", yaxis="Loss",
    labels=["Train", "Test"]
)

## Sanity check

In [21]:
with torch.inference_mode():
    test_logits = model(test_data)
    test_logits = test_logits[:, -1, :]
    test_preds = test_logits.argmax(dim=-1)
    test_acc = (test_preds == test_labels).float().mean()
    print("Test accuracy:", test_acc.item())

Test accuracy: 0.9982500672340393


## Save model

In [22]:
%mkdir ../models

In [23]:
filename = "../models/min_int_model.pt"
torch.save(model.state_dict(), filename)

In [24]:
# sanity check we can load in model
MAX_NUM=200
cfg = HookedTransformerConfig(
    n_layers=1,
    n_heads=1,
    d_model=32,
    d_head=32,
    n_ctx=3, # a b =
    d_vocab=MAX_NUM+1, # 0, 1, ..., MAX_NUM-1, =
    d_vocab_out=MAX_NUM,
    normalization_type=None,
    attn_only=True,
    device=device,
    seed=0
)

model_loaded = HookedTransformer(cfg)
model_loaded.load_state_dict(torch.load(filename))

<All keys matched successfully>